In [20]:
import pandas as pd
import numpy as np
from timeseries_toolkit import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression

In [14]:
df = pd.read_csv('../timeseries_toolkit/tests/fixtures/test_ts_data.csv')

In [16]:
df_rolled = preprocessing.get_rollwin_df(df, feature_window=5, forecast_horizon=3, 
                             df_modelling=True, target_col='target', 
                            datetime_col='datetime', dtformat='%d/%m/%Y')

/Users/alexi/anaconda/envs/timeseries_toolkit/lib/python3.6/site-packages/timeseries_toolkit/preprocessing.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  -forecast_horizon)
/Users/alexi/anaconda/envs/timeseries_toolkit/lib/python3.6/site-packages/tsfresh/utilities/dataframe_functions.py:388: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  differences = df.groupby(grouper)[column_sort].apply(
/Users/alexi/anaconda/envs/timeseries_toolkit/lib/python3.6/site-packages/tsfresh/utilities/dataframe_functions.py:400: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. 

In [17]:
df_rolled

,id,ref_date,winID,datetime,timeID,target_shift,feature_1,feature_2,feature_3,feature_4,target
10,AXP,2017-01-09,5,2017-01-03,1.0,76.88,74.89,75.75,74.74,5853881.0,75.35
11,AXP,2017-01-09,5,2017-01-04,2.0,76.88,75.26,76.55,75.06,4635796.0,76.26
12,AXP,2017-01-09,5,2017-01-05,3.0,76.88,76.00,76.18,74.82,3383003.0,75.32
13,AXP,2017-01-09,5,2017-01-06,4.0,76.88,75.40,75.92,75.06,3089593.0,75.47
14,AXP,2017-01-09,5,2017-01-09,5.0,76.88,76.14,76.50,75.53,4818703.0,75.86
15,AXP,2017-01-10,6,2017-01-04,2.0,76.62,75.26,76.55,75.06,4635796.0,76.26
16,AXP,2017-01-10,6,2017-01-05,3.0,76.62,76.00,76.18,74.82,3383003.0,75.32
17,AXP,2017-01-10,6,2017-01-06,4.0,76.62,75.40,75.92,75.06,3089593.0,75.47
18,AXP,2017-01-10,6,2017-01-09,5.0,76.62,76.14,76.50,75.53,4818703.0,75.86
19,AXP,2017-01-10,6,2017-01-10,6.0,76.62,76.52,78.00,76.12,9561414.0,76.65


In [26]:
aggregations = {'feature_1': 'mean',
                'feature_2': {'mean': 'mean', 'last': 'last'},
                'feature_3': 'max',
                'feature_4': np.median,
                'target': np.std
               }

In [27]:
df_aggregated = preprocessing.get_aggregated_df(df_rolled=df_rolled, 
                                                aggregations=aggregations,
                                                df_modelling=True,
                                                target_col='target_shift',
                                                datetime_col='ref_date'
                                                )

/Users/alexi/anaconda/envs/timeseries_toolkit/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [28]:
df_aggregated

,id,ref_date,feature_1_mean,feature_2_mean,feature_2_last,feature_3_max,feature_4_median,target_std,target_shift,AXP,MMM
0,AXP,2017-01-09,75.538,76.180,76.50,75.53,4635796.0,0.402331,76.88,1,0
1,AXP,2017-01-10,75.864,76.630,78.00,76.12,4635796.0,0.550972,76.62,1,0
2,AXP,2017-01-11,76.146,76.808,77.44,76.12,4500174.0,0.707934,76.60,1,0
3,AXP,2017-01-12,76.312,76.962,76.95,76.12,4500174.0,0.651713,77.49,1,0
4,AXP,2017-01-13,76.618,77.304,77.63,76.41,4500174.0,0.425359,76.69,1,0
5,AXP,2017-01-17,76.870,77.544,77.70,76.47,4500174.0,0.150233,76.20,1,0
6,AXP,2017-01-18,76.988,77.464,77.60,76.51,4500174.0,0.359514,75.97,1,0
7,AXP,2017-01-19,77.176,77.532,77.78,76.61,5988353.0,0.371255,77.43,1,0
8,AXP,2017-01-20,77.008,77.524,76.91,76.61,6450259.0,0.471328,76.89,1,0
9,AXP,2017-01-23,76.806,77.294,76.48,76.61,6450259.0,0.582366,76.93,1,0


In [33]:
features = list(df_aggregated.columns[2:-3]) + list(df_aggregated.columns[-2:])

X = df_aggregated[features].values
y = df_aggregated['target_shift'].values

scaler = MinMaxScaler()

lr = LinearRegression()

X_scaled = scaler.fit_transform(X)

lr.fit(X_scaled, y)